# Transfer Learning with VGG16
**ASL-Translation**

In [11]:
!pip install keras --upgrade

Requirement already up-to-date: keras in /home/ec2-user/anaconda3/envs/theano_p36/lib/python3.6/site-packages (2.2.4)
Requirement not upgraded as not directly required: scipy>=0.14 in /home/ec2-user/anaconda3/envs/theano_p36/lib/python3.6/site-packages (from keras) (1.1.0)
Requirement not upgraded as not directly required: h5py in /home/ec2-user/anaconda3/envs/theano_p36/lib/python3.6/site-packages (from keras) (2.8.0)
Requirement not upgraded as not directly required: keras-preprocessing>=1.0.5 in /home/ec2-user/anaconda3/envs/theano_p36/lib/python3.6/site-packages (from keras) (1.0.5)
Requirement not upgraded as not directly required: numpy>=1.9.1 in /home/ec2-user/anaconda3/envs/theano_p36/lib/python3.6/site-packages (from keras) (1.14.5)
Requirement not upgraded as not directly required: keras-applications>=1.0.6 in /home/ec2-user/anaconda3/envs/theano_p36/lib/python3.6/site-packages (from keras) (1.0.6)
Requirement not upgraded as not directly required: six>=1.9.0 in /home/ec2-use

In [16]:
import keras
print(keras.__version__)

2.2.4


In [17]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [36]:
img_width, img_height = 200, 200
train_data_dir = "train"
validation_data_dir = "test"
nb_train_samples = 2549
nb_validation_samples = 972
batch_size = 56
epochs = 3

In [19]:
vgg_model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0         
__________

In [20]:
# Creating dictionary that maps layer names to the layers
layer_dict = dict([(layer.name, layer) for layer in vgg_model.layers])

# Getting output tensor of the last VGG layer that we want to include
x = layer_dict['block4_pool'].output

In [21]:
# Stacking a new simple convolutional network on top of it    
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(200, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)

In [22]:
# creating the final model 
model_final = Model(input = vgg_model.input, output = predictions)

/home/ec2-user/anaconda3/envs/theano_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=/input_2, outputs=Softmax.0)`
  from ipykernel import kernelapp as app


In [23]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [37]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

Found 2549 images belonging to 2 classes.
Found 972 images belonging to 2 classes.


In [38]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [39]:
# Train the model 
model_final.fit_generator(
    train_generator,
    steps_per_epoch = None,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = None)

Epoch 1/3
46/46 [==============================] - 225s 5s/step - loss: 0.6596 - acc: 0.6058 - val_loss: 0.6972 - val_acc: 0.5257
Epoch 2/3
46/46 [==============================] - 225s 5s/step - loss: 0.6548 - acc: 0.6105 - val_loss: 0.7504 - val_acc: 0.5072
Epoch 3/3
46/46 [==============================] - 225s 5s/step - loss: 0.6400 - acc: 0.6337 - val_loss: 0.6990 - val_acc: 0.5165
